In [1]:
from keras.layers import Activation, Dense
from keras.models import Sequential
from keras.optimizers import SGD, Adam

Using TensorFlow backend.


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import StandardScaler

In [3]:
train = pd.read_csv('data/train.csv', names=list(map(str, range(56))))

In [4]:
train.head()

,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
0,447095,3452,111,22,433,214,3677,252,210,74,...,0,0,0,0,0,0,0,1,0,0
1,113427,3093,95,9,124,7,4115,234,227,124,...,0,0,0,0,0,0,0,0,0,1
2,66435,2551,61,17,90,5,726,231,202,98,...,0,0,0,0,0,0,0,0,0,1
3,8957,2944,135,3,430,13,1868,224,238,149,...,0,0,0,0,0,0,0,0,0,1
4,434631,3030,327,34,277,101,1973,120,181,190,...,0,1,0,0,0,0,0,0,0,1


In [5]:
def normalize(df, cols):
    norms = dict()
    for col in cols:
        norm = StandardScaler()
        norm.fit(np.array(df[col]).reshape(-1,1))
        df[col] = norm.transform(np.array(df[col]).reshape(-1, 1))
        norms[col] = norm
    return df, norms

In [6]:
def dump_csv(model, test_data, ids, filename):
    preds = model.predict(test_data)
    df = pd.DataFrame.from_items([('key', ids), ('score', preds.flatten())])
    df.to_csv(filename, index=False)
    print('CSV saved successfully')

In [7]:
train_normed, norms = normalize(train.copy(), list(map(str, range(1, 11))))

/usr/local/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [8]:
feat_cols = list(map(str, range(1, 55)))
X = train_normed[feat_cols]
y = train_normed['55']

In [9]:
X.head()

,1,2,3,4,5,6,7,8,9,10,...,45,46,47,48,49,50,51,52,53,54
0,1.760238,-0.400681,1.056245,0.767481,2.868112,0.849060,1.490781,-0.675210,-1.792759,1.008587,...,0,0,0,0,0,0,0,0,1,0
1,0.477017,-0.543573,-0.681745,-0.685561,-0.676803,1.129632,0.817984,0.185496,-0.485918,0.717097,...,0,0,0,0,0,0,0,0,0,0
2,-1.460326,-0.847217,0.387787,-0.845442,-0.711054,-1.041279,0.705851,-1.080248,-1.165475,-0.928386,...,0,0,0,0,0,0,0,0,0,0
3,-0.055573,-0.186344,-1.483895,0.753374,-0.574052,-0.309741,0.444208,0.742424,0.167502,-0.590077,...,0,0,0,0,0,0,0,0,0,0
4,0.251828,1.528354,2.660543,0.033907,0.932965,-0.242481,-3.443060,-2.143474,1.239112,-0.217030,...,0,0,1,0,0,0,0,0,0,0


In [10]:
x_train, x_val, y_train, y_val = train_test_split(X, y, random_state=45, test_size=0.2)

In [11]:
x_train.shape

(325367, 54)

In [12]:
model = Sequential()
model.add(Dense(100, input_dim=54))
model.add(Activation('tanh'))
model.add(Dense(50))
model.add(Activation('tanh'))
model.add(Dense(25))
model.add(Activation('tanh'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               5500      
_________________________________________________________________
activation_1 (Activation)    (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                5050      
_________________________________________________________________
activation_2 (Activation)    (None, 50)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 25)                1275      
_________________________________________________________________
activation_3 (Activation)    (None, 25)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 26        
__________

In [14]:
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [23]:
model.fit(x_train.as_matrix(), y_train.as_matrix().reshape(-1, 1), epochs=50, verbose=1, 
          validation_data=(x_val.as_matrix(), y_val.as_matrix()), batch_size=3000)

Train on 325367 samples, validate on 81342 samples
Epoch 1/50
 27000/325367 [=>............................] - ETA: 0s - loss: 0.1785 - acc: 0.9281

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


325367/325367 [==============================] - 1s 3us/step - loss: 0.1760 - acc: 0.9269 - val_loss: 0.1854 - val_acc: 0.9234
Epoch 2/50
325367/325367 [==============================] - 1s 3us/step - loss: 0.1752 - acc: 0.9276 - val_loss: 0.1825 - val_acc: 0.9249
Epoch 3/50
325367/325367 [==============================] - 1s 3us/step - loss: 0.1745 - acc: 0.9278 - val_loss: 0.1814 - val_acc: 0.9255
Epoch 4/50
325367/325367 [==============================] - 1s 3us/step - loss: 0.1739 - acc: 0.9281 - val_loss: 0.1811 - val_acc: 0.9253
Epoch 5/50
325367/325367 [==============================] - 1s 3us/step - loss: 0.1737 - acc: 0.9279 - val_loss: 0.1816 - val_acc: 0.9243
Epoch 6/50
325367/325367 [==============================] - 1s 3us/step - loss: 0.1726 - acc: 0.9283 - val_loss: 0.1794 - val_acc: 0.9267
Epoch 7/50
325367/325367 [==============================] - 1s 3us/step - loss: 0.1719 - acc: 0.9288 - val_loss: 0.1794 - val_acc: 0.9265
Epoch 8/50
325367/325367 [===================

In [24]:
model.save('FeedforwardNet-03-0.93.h5py')

In [17]:
test = pd.read_csv('data/test.csv', names=list(map(str, range(55))))

In [19]:
for col in norms:
    test[col] = norms[col].transform(np.array(test[col]).reshape(-1, 1))

/usr/local/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [20]:
test.head()

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
0,258918,-2.171638,1.224710,1.189936,-0.064844,-0.026046,-1.377581,-2.284355,0.236126,1.892532,...,0,0,0,0,0,0,0,0,0,0
1,258134,-0.606036,-0.606088,-0.815437,0.283134,-0.197298,-0.609531,0.743229,0.185496,-0.381371,...,0,0,0,0,0,0,0,0,0,0
2,165265,-0.477357,-0.364958,-0.949128,-0.506869,-0.385675,-0.852949,0.743229,0.489275,-0.224550,...,0,0,0,0,0,0,0,0,0,0
3,522141,0.981012,-0.168483,0.254095,-0.563298,-0.454176,-0.886259,1.154382,0.438645,-0.799560,...,0,0,0,0,0,0,0,0,0,0
4,569662,-0.534547,-0.874010,1.056245,-0.685561,-0.368550,0.603079,0.668474,-1.789065,-1.635938,...,0,0,0,0,0,0,0,0,0,0


In [21]:
x_test = test[feat_cols]

In [25]:
dump_csv(model, x_test.as_matrix(), test['0'].as_matrix(), 'Experiment-05-FFN.csv')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  This is separate from the ipykernel package so we can avoid doing imports until


CSV saved successfully


In [22]:
model.save('FeedforwardNet-01.h5py')

In [23]:
predictions = model.predict(x_test.as_matrix())

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


array([0.02121277, 0.9571678 , 0.8251506 , ..., 0.5365085 , 0.93746066,
       0.02533947], dtype=float32)